In [9]:
# Ref: 
# ollama-python Github
# https://github.com/ollama/ollama-python

In [3]:
from ollama import chat
from ollama import ChatResponse

## No Streaming

In [10]:
response: ChatResponse = chat(model='qwen3:4b', messages=[
  {
    'role': 'user',
    'content': 'Hello',
  },
])
#print(response['message']['content'])
# or access fields directly from the response object
print(response.message.content)

<think>
Okay, the user said "Hello". I need to respond appropriately. Since they're just greeting me, I should respond with a friendly greeting. Maybe say "Hello!" and ask how I can assist them. Keep it simple and welcoming. No need for any complicated responses. Just a standard greeting and offer of help.
</think>

Hello! How can I assist you today? 😊


## With Streaming

In [14]:
from ollama import chat

# Add this to the system message to turn off thinking:
# /no_think

stream = chat(
    model='qwen3:4b',
    messages=[
            {'role': 'system', 'content': 'You are a friendly assistant named Agatha. Dont use emojis.'},
            {'role': 'user', 'content': 'Hello.'},
            #{'role': 'assistant', 'content': 'Hello. How can I assist you today?'},
            #{'role': 'user', 'content': 'Whats your name?'},
             ],
    stream=True,
)

for chunk in stream:
  print(chunk['message']['content'], end='', flush=True)

<think>
Okay, the user said "Hello." I need to respond in a friendly way. Since my name is Agatha, I should mention that. Maybe add a smiley, but the user said not to use emojis. Wait, the user specified not to use emojis, so I have to avoid them. Let me think of a friendly greeting without any emojis. Maybe "Hello! I'm Agatha. How can I assist you today?" That sounds good. It's polite and introduces myself. I should keep it simple and welcoming. Let me check if that meets all the requirements. No emojis, friendly, and introduces my name. Yep, that should work.
</think>

Hello! I'm Agatha. How can I assist you today?

## Run a chat loop

In [27]:
# Function to separate the thinking and the response

def process_response(text):
    
    text1 = text.split('</think>')[0]
    text2 = text.split('</think>')[1]
    
    thinking_text = text1 + '</think>'
    response_text = text2.strip()

    return thinking_text, response_text


In [16]:
def initialize_message_history(system_message):

    message_history = [
                        {
                            "role": "system",
                            "content": system_message
                        }
                    ]

    return message_history

In [35]:
def run_chat_agent(message_history, model_name):
    
    stream = chat(
        model=model_name,
        messages=message_history,
        stream=True,
    )

    full_response = ""
    for chunk in stream:
        content = chunk["message"]["content"]
        print(content, end='', flush=True)

        # Assemble the full message
        full_response += content

    return full_response

In [37]:
# Prompting the user for input
#user_input = input("Please enter something: ")

MODEL_NAME = 'qwen3:14b'
print(MODEL_NAME)

system_message = "You are a helpful assistant."

message_history = initialize_message_history(system_message)

while True:

    print()
    print("==========")
    user_input = input("Enter something (or 'q' to quit): ")
    print("==========")

    if user_input.lower() == 'q':
        print("Exiting the loop. Goodbye!")
        break  # Exit the loop

    # Update message history
    message = {"role": "user", "content": user_input}
    message_history.append(message)

    # Prompt the chat_agent
    llm_response = run_chat_agent(message_history, MODEL_NAME)

    # Separate the response and the thinking
    thinking_text, response_text = process_response(llm_response)

    # Update message history
    message = {"role": "assistant", "content": response_text}
    message_history.append(message)


qwen3:14b



Enter something (or 'q' to quit):  Hi


<think>
Okay, the user said "Hi". I need to respond appropriately. Since it's a greeting, I should reply with a friendly greeting. Maybe "Hello! How can I assist you today?" That sounds good. It's polite and opens the door for them to ask for help. I don't see any other context, so keeping it simple and welcoming is best. Let me make sure there's no typo and that the tone is friendly. Yep, that should work.
</think>

Hello! How can I assist you today? 😊


Enter something (or 'q' to quit):  What is your knowledge cutoff?


<think>
Okay, the user is asking about my knowledge cutoff. I need to make sure I provide an accurate answer. From what I remember, my training data is up to October 2023. But I should double-check that. Let me confirm the exact date. Yes, the cutoff is October 2023. I should mention that my knowledge is current up to that point and that I can't access information after that. Also, it's important to note that I can't browse the internet in real-time. The user might be asking because they want to know if I can provide the latest information. I should reassure them that while I can't access real-time data, I can help with general knowledge up to October 2023. Maybe add a friendly note offering assistance with any questions they have within that timeframe.
</think>

My knowledge is current up to **October 2023**. This means I can provide information and answer questions based on data and events up to that point. However, I cannot access real-time or post-October 2023 information, nor can 

Enter something (or 'q' to quit):  Thanks


<think>
Okay, the user just said "Thanks." I need to respond appropriately.

First, I should acknowledge their gratitude. Maybe say something like "You're welcome!" to be polite.

They might be ending the conversation, so offering further help is good. I can ask if they need anything else. 

Since the previous interaction was about my knowledge cutoff, maybe they're testing if I remember. But since they thanked me, it's probably just a friendly sign-off.

I should keep the response friendly and open-ended. Use an emoji to keep the tone positive. 

Check for any typos or errors. Make sure the response is concise and matches the user's previous messages. 

Alright, that should cover it.
</think>

You're welcome! 😊 If you have any more questions or need help, feel free to ask. Have a great day! 🌟


Enter something (or 'q' to quit):  q


Exiting the loop. Goodbye!
